In [1]:
import json
import foursquareWithHosts
with open('keys/appCredentialsSudo.json') as credentialsFile:    
    credentials = json.load(credentialsFile)
with open('keys/hosts.json') as hostsFile:    
    hosts = json.load(hostsFile)
with open('keys/accessTokensSudo.json') as tokensFile:    
    accessTokens = json.load(tokensFile)

In [ ]:
clients = []
users = []
for accessToken in accessTokens[:3]:
    client = foursquareWithHosts.Foursquare(client_id=credentials['client_id'], client_secret=credentials['client_secret'], redirect_uri=credentials['redirect_uri'])
    client.set_access_token(accessToken)
    user = client.users()
    clients.append(client)
    users.append(user)

In [ ]:
auth_uri = client.oauth.auth_url()

In [ ]:
# Interrogate foursquare's servers to get the user's access_token
access_token = client.oauth.get_token('code')

# Apply the returned access token to the client
client.set_access_token(access_token)

# Get the user's data
user = client.users()

In [ ]:
client.set_access_token(accessTokens[1])

# Get the user's data
user = client.users()

In [ ]:
#1 Get the venues of interes
yogaVenues = client.venues.search(params={'query': 'worldclass', 'near':'Moscow'})
for yogaPlace in yogaVenues['venues']:
    if yogaPlace['hereNow']['count'] > 0:
        print ''
        print yogaPlace['name']
        print yogaPlace['id']
        print 'here now:', yogaPlace['hereNow']['count']
        print 'total users:', yogaPlace['stats']['usersCount']

In [35]:
print yogaVenues['venues'][0].keys()

[u'verified', u'name', u'referralId', u'hereNow', u'specials', u'contact', u'location', u'stats', u'id', u'categories']


In [14]:
#Save info about Moscow
with open('moscowGeocode.json', 'w') as outfile:
    json.dump(yogaVenues['geocode'], outfile)

In [ ]:
#2 Check-in a user into the venue of interest
clients[0].checkins.add({'venueId':venuesForUsers[0][0]})

In [ ]:
#3 Get info about the venue right now
hereNowDict = clients[0].venues.herenow(venuesForUsers[0][0])

# Getting the data 

In [18]:
import pandas as pd
usersDF = pd.DataFrame()
checkinsDF = pd.DataFrame()
checkpointsDF = pd.DataFrame()
allUsersIds = []
allCheckInsIds = []

currentVenues = [0,0,0]
currentBotId = 0

In [19]:
#3.1 Savethe info to the DB
def parseHereNowDict(hereNowDict, venueId):
    usersInVenue = []
    currentCheckIns = []

    if hereNowDict['hereNow']:
        global checkpointsDF
        global usersDF
        global checkinsDF
        hereNowCount =  hereNowDict['hereNow']['count']
        checkpointsDF = checkpointsDF.append([{'createdAt':datetime.now(),
                                               'hereNow':hereNowCount, 
                                               'venueId':venueId}])

        print 'Here now:', hereNowCount
        print ''

        #Get check-in info first
        for checkinAtVenue in hereNowDict['hereNow']['items']:
            if checkinAtVenue['type'] == 'checkin':
                if checkinAtVenue['id'] not in allCheckInsIds:
                    currentCheckIns.append({'checkInId':checkinAtVenue['id'],
                                            'createdAt':checkinAtVenue['createdAt'],
                                            'createdAtDate':extractComponents(checkinAtVenue['createdAt'],checkinAtVenue['timeZoneOffset']),
                                            'venueId':venueId,
                                            'userId':checkinAtVenue['user']['id']
                                           })
                    allCheckInsIds.append(checkinAtVenue['id'])
                    #Now get the user
                    userInVenue =  checkinAtVenue['user']
                    #print 'User', userInVenue['id'],'Checked-in at:', extractComponents(checkinAtVenue['createdAt'],checkinAtVenue['timeZoneOffset'])
                    if userInVenue['id'] not in allUsersIds:
                        userInfo = {'userId':userInVenue['id']}
                        if 'gender' in userInVenue.keys() and userInVenue['gender']:
                            userInfo["gender"] = userInVenue['gender']
                        if 'firstName' in userInVenue.keys() and userInVenue['firstName']:
                            userInfo["firstName"] = userInVenue['firstName']
                        if 'lastName' in userInVenue.keys() and userInVenue['lastName']:
                            userInfo["lastName"] = userInVenue['lastName']
                        usersInVenue.append(userInfo)
                        allUsersIds.append(userInVenue['id'])
        if len(usersInVenue):
            usersDF = usersDF.append(usersInVenue)
        if len(currentCheckIns):
            checkinsDF = checkinsDF.append(currentCheckIns)

        usersDF.to_csv('data/realUsers.tsv', sep='\t',encoding='utf-8')
        checkinsDF.to_csv('data/realCheckins.tsv', sep='\t',encoding='utf-8')
        checkpointsDF.to_csv('data/checkPoints.tsv', sep='\t',encoding='utf-8')

In [20]:
def getNextCheckinInfo():
    global currentBotId
    global currentVenues
    currentVenue = venuesForUsers[currentBotId][currentVenues[currentBotId]]
    checkIn = clients[currentBotId].checkins.add(currentVenue)
    hereNowDict = clients[currentBotId].venues.herenow(currentVenue['venueId'])
    parseHereNowDict(hereNowDict, currentVenue['venueId'])
    
    currentVenues[currentBotId] = (currentVenues[currentBotId]+1)%len(venuesForUsers[currentBotId])
    currentBotId = (currentBotId+1)%len(clients)
    print currentBotId
    print currentVenues

In [21]:
with open('data/top90healthVenuesSummer.json') as venuesFile:
    top50HealtPlaces = json.load(venuesFile)
    
venuesForUsers = [[],[],[]]
for i, x in enumerate(top50HealtPlaces):
    venuesForUsers[i%3].append(x)

In [22]:
from datetime import datetime
def extractComponents(createdAt, tzOffset):
    date = datetime.utcfromtimestamp(createdAt+(tzOffset-60)*60)
    return date

In [8]:
timeInterval = 3*60/(len(venuesForUsers[0])*len(clients))

In [ ]:
import schedule
import time

timeInterval = 3*60/(len(venuesForUsers[0])*len(clients))
schedule.every(1).minutes.do(getNextCheckinInfo)

while True:
    schedule.run_pending()
    time.sleep(1)

In [32]:
schedule.every(timeInterval).minutes.do(getNextCheckinInfo)  & checkinsDF.userId!=128636353 and checkinsDF.userId!=130936370

Every 6 minutes do getNextCheckinInfo() (last run: [never], next run: 2015-06-23 18:12:28)

In [45]:
len(checkinsDF1)

339

In [44]:
checkinsDF1.to_csv('data/realCheckins.tsv', sep='\t',encoding='utf-8')

In [27]:
def job():
    print("I'm working...")

schedule.every(10).minutes.do(job)

Every 10 minutes do job() (last run: [never], next run: 2015-06-23 18:15:24)

In [73]:
requestedUsersFriends = []
requestedUsers = []

In [ ]:
#4 Get the user that was checked-in
requestedUser = client.users('someUserId')

#5 Send requests to their friends 
if requestedUser['user']:
    requestedUserInfo = requestedUser['user']
    print 'Blocked:', requestedUserInfo['blockedStatus']
    print requestedUserInfo['homeCity']
    print requestedUserInfo['checkins']
    print requestedUserInfo['gender']
    print 'friends count:', requestedUserInfo['friends']['count']
    for friendGroup in requestedUserInfo['friends']['groups']:
        if friendGroup['type'] == 'friends':
            if friendGroup['count'] > 3:
                print 'Enough mutual friends'
                break
        if 0:
            print 'Already requested'
        print ''
        for friend in friendGroup['items']:
            if not friend['id'] in requestedUsersFriends:
                print 'Add'
                requestedUsersFriends.append(friend['id'])
                client.users.request(friend['id'])
    #0 for Mutual friends, type = 'friends', 1 for other friends, type = 'others'
    #print requestedUserInfo['friends']['groups']
    #print requestedUser['user']['lists']
     #print requestedUser['user']['tips']

In [66]:
requestedUser['user'].keys()

[u'checkins',
 u'bio',
 u'gender',
 u'type',
 u'firstName',
 u'lastName',
 u'blockedStatus',
 u'pings',
 u'homeCity',
 u'photos',
 u'contact',
 u'checkinPings',
 u'lists',
 u'photo',
 u'tips',
 u'mayorships',
 u'friends',
 u'id',
 u'badges']

In [179]:
print user['user']['friends']['groups'][1]

{u'count': 0, u'items': [], u'type': u'others', u'name': u'Other friends'}


In [128]:
# Get the user's data
user = client.users()

In [ ]:
#6 One way to get some check-in
aFriend = client.users('')
print aFriend['user']['checkins']['items'][0]

In [180]:
#6 Another way — get recent check-ins
recent = client.checkins.recent()['recent']

In [184]:
recent

[]

In [119]:
#Doesn't work now:(
someCheckIns = client.users.all_checkins('')

In [148]:
requestedUsers

[]

In [23]:
user = client.users()

In [ ]:
client.users.requests()